# Data Preparation Notebook
This notebook transforms the source dataset into a single dataset that can be used to train machine learning models.

First, we'll perform data munging on our data to get it into an appropriate shape.

After that is complete, we'll use the Azure ML SDK to upload this data to Azure as a registered dataset.

### Overview
I am using the [NHL Game Data dataset](https://www.kaggle.com/datasets/martinellis/nhl-game-data) from Kaggle.com compiled by Martin Ellis which contains a series of CSV files containing NHL hockey game and player data from 2001 to 2020.

![Kaggle Dataset](screens/Kaggle.png)

## Data Munging
Our source dataset is in many different files. We need to join various files together and aggregate down to game-level information before we can use it for training.

The dataset is stored in multiple CSV files structured like a relational database. In fact, the author of the dataset was so kind as to provide this schema diagram of the dataset below:

![Schema Diagram](data/table_relationships.JPG)

In [1]:
# Import Pandas so we can work with the data
import pandas as pd

# Read the data out of the source file
df_team_stats = pd.read_csv('data/game_teams_stats.csv')

# We only care about the game and the penalties in minutes, so aggregate and simplify
df_team_stats = df_team_stats[['game_id', 'pim']]
df_team_stats.rename(columns={'pim': 'penaltyMinutes'}, inplace=True)
df_team_stats = df_team_stats.groupby('game_id').agg('sum')

# Display the top 5 records
df_team_stats.head()

,penaltyMinutes
game_id,
2000020001,42.0
2000020002,32.0
2000020003,45.0
2000020004,24.0
2000020005,52.0


In [2]:
# Load Game Data
df_games = pd.read_csv('data/game.csv')

# Remove unnecessary columns
df_games = df_games[['game_id', 'season', 'type', 'away_team_id', 'home_team_id']]

# Historical information is helpful, but does become outdated. Let's look at 2015 onwards, then drop our identifiers that might throw off machine learning
df_games = df_games[df_games['season'] >= 20152016]

# Remap the season to a string type
df_games['season'] = df_games['season'].map(str)

# Display the first 5 results
df_games.head()

,game_id,season,type,away_team_id,home_team_id
0,2016020045,20162017,R,4,16
1,2017020812,20172018,R,24,7
2,2015020314,20152016,R,21,52
3,2015020849,20152016,R,52,12
4,2017020586,20172018,R,20,24


In [3]:
# Aggregate everything together
df_detailed_pens = pd.merge(left=df_team_stats, right=df_games, how='left', left_on='game_id', right_on='game_id')

df_detailed_pens.head()

,game_id,penaltyMinutes,season,type,away_team_id,home_team_id
0,2000020001,42.0,NaN,NaN,NaN,NaN
1,2000020002,32.0,NaN,NaN,NaN,NaN
2,2000020003,45.0,NaN,NaN,NaN,NaN
3,2000020004,24.0,NaN,NaN,NaN,NaN
4,2000020005,52.0,NaN,NaN,NaN,NaN


In [4]:
# We need to know a bit more about who these teams are. Load a lookup table
df_teams = pd.read_csv('data/team_info.csv')[['team_id', 'teamName']]
df_teams.head()

,team_id,teamName
0,1,Devils
1,4,Flyers
2,26,Kings
3,14,Lightning
4,6,Bruins


In [5]:
# Now get the home and away teams into the detailed penalties dataframe
df_detailed_pens = pd.merge(left=df_detailed_pens, right=df_teams, how='inner', left_on='home_team_id', right_on='team_id')
df_detailed_pens = df_detailed_pens.rename(columns={'teamName': 'homeTeam'})
df_detailed_pens = df_detailed_pens.drop(['home_team_id','team_id'], axis=1)

df_detailed_pens = pd.merge(left=df_detailed_pens, right=df_teams, how='inner', left_on='away_team_id', right_on='team_id')
df_detailed_pens = df_detailed_pens.rename(columns={'teamName': 'awayTeam'})
df_detailed_pens = df_detailed_pens.drop(['away_team_id','team_id'], axis=1)

df_detailed_pens.head()

,game_id,penaltyMinutes,season,type,homeTeam,awayTeam
0,2015020001,12.0,20152016,R,Maple Leafs,Canadiens
1,2015020712,12.0,20152016,R,Maple Leafs,Canadiens
2,2016020594,76.0,20162017,R,Maple Leafs,Canadiens
3,2016020907,12.0,20162017,R,Maple Leafs,Canadiens
4,2017021101,10.0,20172018,R,Maple Leafs,Canadiens


## Data Visualization
Because it's interesting to view relationships in our data, let's look at the distributions of the penalties in minutes column

In [6]:
# For fun, let's look at descriptive statistics for penalty minutes
df_detailed_pens.penaltyMinutes.describe()


count    9142.000000
mean       27.120543
std        20.357195
min         0.000000
25%        14.000000
50%        22.000000
75%        32.000000
max       208.000000
Name: penaltyMinutes, dtype: float64

In [8]:
import plotly.express as px

# Common variables for charts
plotly_settings = {
    'hover_data': ['season','type', 'homeTeam', 'awayTeam'],
    'labels': {'penaltyMinutes': 'Penalty Minutes', 'season': 'Season', 'type': 'Game Type', 'homeTeam': 'Home Team', 'awayTeam': 'Away Team'},
     'color_discrete_sequence': px.colors.sequential.Agsunset
}

In [19]:
# Let's plot it out as a Box Plot over seasons
px.box(df_detailed_pens, x='penaltyMinutes', y='season', color='season', title='Distribution of Penalty Minutes', **plotly_settings).show()

In [13]:
# Let's draw a histogram with the same data
px.histogram(df_detailed_pens, x='penaltyMinutes', title='Distribution of Penalty Minutes', color='season', marginal='violin', **plotly_settings).show()

## Registering a Data Set

Now that we have a candidate data set, let's drop irrelevant columns and register the result on Azure as a dataset

In [27]:
# Drop unnecessary columns
df_training = df_detailed_pens.drop(columns=['season', 'game_id'])
df_training.head()

,penaltyMinutes,type,homeTeam,awayTeam
0,12.0,R,Maple Leafs,Canadiens
1,12.0,R,Maple Leafs,Canadiens
2,76.0,R,Maple Leafs,Canadiens
3,12.0,R,Maple Leafs,Canadiens
4,10.0,R,Maple Leafs,Canadiens


In [28]:
# Connect to the workspace by reading from the config.json file (downloaded from the Machine Learning page in the portal)
from azureml.core import Workspace, Dataset


ws = Workspace.from_config() # config.json is needed in this folder for this to work

# Display high-level info on the workspace
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: DataScience
Azure region: northcentralus
Resource group: datascience


In [39]:
# The default datastore is a blob storage container where datasets are stored
datastore = ws.get_default_datastore()

# Register the dataset
ds = Dataset.Tabular.register_pandas_dataframe(
        dataframe=df_training, 
        name='NHL-Penalties-2020', 
        description='A breakdown of penalty minutes per game matchup',
        target=datastore
    )

# Display information about the dataset
print(ds.name + " v" + str(ds.version) + ' (ID: ' + ds.id + ")")

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/783d4c3f-e99e-47b9-ac1d-df55a2564c72/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.
NHL-Penalties-2020 v6 (ID: 69058fd8-167e-4e66-9aa8-733590aa2e4c)
